# Use TF-IDF to identify (possibly) irrelevant papers

In [12]:
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('../../coding_website/db_llm_education_survey.sqlite3')

# Create a cursor object
cursor = conn.cursor()

# Execute a SQL query
cursor.execute('SELECT llm_education_survey_paper.id, title, abstract, source FROM llm_education_survey_paper JOIN '
               'llm_education_survey_analysis ON llm_education_survey_paper.id = llm_education_survey_analysis.paper_id '
               'WHERE is_relevant = 1 AND user_id = 1')

# fetch all the results in a data frame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=['id', 'title','abstract', 'source'])

print(df.shape)

# Combine relevant text columns for vectorization
df['combined_text'] = df[['title', 'abstract']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Define query related to LLMs and CS education
query = "Large Language Models and machine learning applied to Computer Science education and teaching. Pedagogies and methodologies for teaching computer science students using LLMs."

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
query_vec = vectorizer.transform([query])

# Calculate cosine similarity between the query and all documents
cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

# Get top N relevant papers
N = 500
top_n_indices = cosine_similarities.argsort()[-N:][::-1]
relevant_papers = df.iloc[top_n_indices]

# get least relevant papers
least_n_indices = cosine_similarities.argsort()[:100]
least_relevant_papers = df.iloc[least_n_indices]
# exclude papers with terms like "call for papers" or "conference"
least_relevant_papers = least_relevant_papers[~least_relevant_papers['combined_text'].str.contains('teach|student|educat|pedagog|classroom|course|CS1', case=False)]


least_relevant_papers


(187, 4)


,id,title,abstract,source,combined_text
7,50,PapagAI: Automated Feedback for Reflective Essays,None,springer,PapagAI: Automated Feedback for Reflective Ess...
9,54,Comparative Quality Analysis of GPT-Based Mult...,None,springer,Comparative Quality Analysis of GPT-Based Mult...
37,221,Text Analysis on Early Reactions to ChatGPT as...,None,springer,Text Analysis on Early Reactions to ChatGPT as...
56,376,Studying the effect of AI Code Generators on S...,AI code generators like OpenAI Codex have the ...,"acm, scopus",Studying the effect of AI Code Generators on S...
73,487,A Large-Scale Study of ML-Related Python Projects,The rise of machine learning (ML) for solving ...,acm,A Large-Scale Study of ML-Related Python Proje...
8,51,Large Language Model Assisted Software Enginee...,None,springer,Large Language Model Assisted Software Enginee...
1,32,Learner Models for MOOC in a Lifelong Learning...,None,springer,Learner Models for MOOC in a Lifelong Learning...


# Export all papers to CSV for stage 1 analysis
Get a dump of papers' metadata (title and abstract) for stage 1 analysis.

# Create a cursor object
cursor = conn.cursor()

# Execute a SQL query
cursor.execute('SELECT llm_education_survey_paper.id, title, abstract, source, is_relevant FROM llm_education_survey_paper JOIN '
               'llm_education_survey_analysis ON llm_education_survey_paper.id = llm_education_survey_analysis.paper_id '
               'WHERE user_id = "1"')

# fetch all the results in a data frame
df_all = pd.DataFrame(cursor.fetchall(), columns=['id', 'title','abstract', 'source', 'is_relevant'])
# change data type of is_relevant to string
df_all['is_relevant'] = df_all['is_relevant'].astype(str)
# fill na as empty string

df_all.to_csv('../data/stage1_all_papers.csv', index=False)
